# 分析尝试

目的：  
（1）判断出即将大幅上涨的币；  
（2）判断出即将大幅下跌的币；  
（3）配对交易，做无风险套利

In [1]:
import os
from DataRead import Read, Analysis
from DataQuery import Ticker, Kline, Query
import pandas as pd
import numpy as np
import time
import requests
from Tactic import MACD, SMA, MultiTacs, Tactic
import datetime
# import talib
%matplotlib inline
import time
from pyecharts.charts import Line
import datetime
from itertools import *
from statsmodels.tsa.stattools import adfuller as ADF
from tqdm import tqdm
import math

In [4]:
start='2023-1-1'
end='2023-7-1'
interval='4h'
num=20
threshold=0.0015
pair=['BTCUSDT','ETHUSDT']
analysis=Analysis()
res=analysis.return_diff(market_pair=pair,interval=interval,start=start,end=end)

In [5]:
res

{'market_1': 'BTCUSDT',
 'market_2': 'ETHUSDT',
 'return_diff': 0.000131,
 'daily_diff': 0.000785,
 'p_value': 0.0}

In [ ]:
start='2022-1-1'
end='2023-6-18'
line2=analysis.lineplot_individual_returns(markets=['TRXUSDT','JASMYUSDT'],interval='3d',start=start,end=end)
line2.render_notebook()

In [ ]:
analysis=Analysis()
markets=['BTCUSDT','ETHUSDT']
start='2023-1-1'
end='2023-7-11'
interval='1d'
merge_return=analysis.merge_data_for_markets(markets=markets,start=start,end=end,interval=interval,col='Return')
merge_return['diff']=merge_return[markets[0][:-4]]-merge_return[markets[1][:-4]]
merge_return.dropna(subset=['diff'],inplace=True)

line=analysis.pyecharts_line_plot(df=merge_return[['diff']])
line.render_notebook()

In [ ]:
ADF(merge_return['diff'])

In [ ]:
start='2020-1-1'
end='2023-6-18'
line=analysis.plot_price_to_bench(markets=['ETHUSDT'],interval='1d',start=start,end=end,scaled=False)
line.render_notebook()

In [ ]:
markets=['ETHUSDT', 'BTCUSDT', 'DOGEUSDT']
start='2023-1-1'
end='2023-6-17'
window=1
# line=analysis.plot_price_volatility(markets=markets, interval='3d',start=start, end=end)
line=analysis.plot_high_above_close(markets=markets, interval='1d',start=start, end=end,window=window)
line.render_notebook()

In [ ]:
read=Read()
eth=read.read_kline(market='ETHUSDT',freq_type='1d')['close']
btc=read.read_kline(market='BTCUSDT',freq_type='1d')['close']
eth=eth.loc[start:end].copy()
btc=btc.loc[start:end].copy()
df=eth/btc

In [ ]:
df.plot()

最近一个月表现最好的货币

In [ ]:
start='2023-2-1'
end='2023-2-28'
best10=analysis.best_performed_of_last_n(interval='1d',start=start, end=end)

In [ ]:
best10

In [ ]:
tgt_mkts = best10['symbol'].values
start='2023-1-1'
end='2023-6-10'
line=analysis.lineplot_accumulative_returns(markets=tgt_mkts,interval='1d',start=start,end=end)
line.render_notebook()

In [ ]:
line=analysis.lineplot_accumulative_returns(markets=tgt_mkts,interval='1d',start=start,end=end)
line.render_notebook()

In [ ]:
start='2023-1-1'
end='2023-12-31'
market_shares = analysis.turnover_value_share(start=start,end=end)
big10=market_shares.iloc[:10]

In [ ]:
start='2023-1-1'
end='2023-6-10'
big10_markets=big10['symbol'].values
line_big10=analysis.lineplot_accumulative_returns(markets=big10_markets,interval='1d',start=start, end=end)
line_big10.render_notebook()

2023年1月表现最好货币，在接下来的表现：

计算成交额比重

In [ ]:
sma=talib.SMA(close,timeperiod=12)

In [ ]:
sma_ob=SMA()
df2=sma_ob.sma_algorithm(df=df_kline)

In [ ]:
s_date='2017-10-1'
today=datetime.datetime.today()
e_date=today.strftime('%Y-%m-%d')
short_flag = True
tacs=['SMA','DEA','EMA']

result=pd.DataFrame()
for i in tqdm(range(0,len(usdt_markets))):
    mkt = usdt_markets[i]
    read=Read()
    try:
        path = os.path.join('/Users/zhengye/PycharmProjects/BINANCE/tradeData/kline/1d',mkt+'-1d.json')
        if not os.path.exists(path):
            continue
        df_kline=read.read_kline(freq_type='1d',market=mkt)
        multi_tacs=MultiTacs()
        df_multi_tac_returns=multi_tacs.multi_tac_returns(df_kline,tacs=tacs, start=s_date, end=e_date,short_flag=short_flag)
        df_multi_tac_returns['symbol']=mkt[:-4]
        # result=result.append(df_multi_tac_returns)
        result=pd.concat([result, df_multi_tac_returns.iloc[[0],:]],ignore_index=True)
    except:
        print(mkt)
        continue
reuslt=result.sort_values(by='strategy_diff',ascending=False)

In [ ]:
macd, macdsignal, macdhist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)

In [ ]:
analysis.top_n_percent_change_markets()

### 分析一、  
从2020年1月开始，每个月收益表现最好的币，在接下来的1个月、3个月、6个月、1年、至今的表现

In [ ]:
best_of_202001=analysis.best_performed_of_last_n(n=30,interval='1d',end='20200131',num=10)

In [ ]:
start='2023-1-1'
start=datetime.datetime.strptime(start,'%Y-%m-%d')
two_weeks_after=(start+datetime.timedelta(weeks=2)).strftime('%Y-%m-%d')
one_month_after=(start+datetime.timedelta(days=30)).strftime('%Y-%m-%d')
three_month_after=(start+datetime.timedelta(days=30*3)).strftime('%Y-%m-%d')
six_month_after=(start+datetime.timedelta(days=30*6)).strftime('%Y-%m-%d')

start=start
end=two_weeks_after
best7_mkts=analysis.best_performed_of_last_n(interval='1d',start=start,end=end,num=7)['symbol'].values

for end_date in [one_month_after, three_month_after, six_month_after]:
    merge=analysis.merge_data_for_markets(markets=best7_mkts,interval='1d',start=start,end=end_date,col='AccumReturn')
    print(merge)